In [ ]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedKFold

le = LabelEncoder()

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving drebin.csv to drebin.csv


In [ ]:
data = pd.read_csv('drebin.csv')

data.head()

<ipython-input-3-f6093674d2f4>:1: DtypeWarning: Columns (92) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('drebin.csv')


,transact,onServiceConnected,bindService,attachInterface,ServiceConnection,android.os.Binder,SEND_SMS,Ljava.lang.Class.getCanonicalName,Ljava.lang.Class.getMethods,Ljava.lang.Class.cast,...,READ_CONTACTS,DEVICE_POWER,HARDWARE_TEST,ACCESS_WIFI_STATE,WRITE_EXTERNAL_STORAGE,ACCESS_FINE_LOCATION,SET_WALLPAPER_HINTS,SET_PREFERRED_APPLICATIONS,WRITE_SECURE_SETTINGS,class
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,S
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,S
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,S
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,1,1,0,0,0,S
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,S


In [ ]:
data.fillna(value=pd.np.nan, inplace=True)

data = data[(data.astype(str) != '?').all(axis=1)]

data = data.reset_index(drop = True)

<ipython-input-4-78e8e81d846f>:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  data.fillna(value=pd.np.nan, inplace=True)


In [ ]:
y = data.iloc[:,-1:]

y.head()

,class
0,S
1,S
2,S
3,S
4,S


In [ ]:
y = le.fit_transform(y)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
X = data.iloc[:,:-1]

X = np.array(X)


In [ ]:
from sklearn.linear_model import LogisticRegression

from sklearn import metrics

In [ ]:

def mal_detection_LR(Xtrain, Xtest, ytrain, ytest):

  LR_classifier = LogisticRegression()

  LR_classifier.fit(Xtrain, ytrain) # train the model

  pred_prob = LR_classifier.predict_proba(Xtest)[:,1] #predict test data

  pred = (pred_prob > 0.5) # using threshold

  fbeta = metrics.fbeta_score(ytest, pred, average='weighted', beta = 10)

  tn, fp, fn, tp = metrics.confusion_matrix(ytest, pred).ravel()

  tpr = tp/(tp+fn)

  fpr = fp/(fp+tn)

  return fbeta, tpr, fpr

In [ ]:
kfold = StratifiedKFold(n_splits= 5, shuffle=True, random_state = 42)

cvscores_LR = []

for train, test in kfold.split(X, y):

  cvscores_LR.append(mal_detection_LR(Xtrain = X[train], Xtest = X[test], ytrain= y[train], ytest = y[test]))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
cvscores_LR

[(0.9787160100000858, 0.9684968496849685, 0.01529535864978903),
 (0.9757126755121599, 0.9558955895589559, 0.012664907651715039),
 (0.9757132789688729, 0.9576957695769577, 0.013720316622691292),
 (0.9770458635618675, 0.9684968496849685, 0.017941952506596307),
 (0.977709879180814, 0.9621962196219622, 0.013192612137203167)]

In [ ]:
pd.DataFrame(cvscores_LR, columns= ['F-beta Score', 'TPR', "FPR"])

,F-beta Score,TPR,FPR
0,0.978716,0.968497,0.015295
1,0.975713,0.955896,0.012665
2,0.975713,0.957696,0.013720
3,0.977046,0.968497,0.017942
4,0.977710,0.962196,0.013193


In [ ]:
def mal_detection_SVM(Xtrain, Xtest, ytrain, ytest):

    SVM_classifier = SVC(kernel = 'rbf', probability = True)

    SVM_classifier.fit(Xtrain, ytrain) 

    pred_prob = SVM_classifier.predict_proba(Xtest)[:,1]

    pred = (pred_prob > 0.5)

    fbeta = metrics.fbeta_score(ytest, pred, average='weighted', beta = 10)

    tn, fp, fn, tp = metrics.confusion_matrix(ytest, pred).ravel()

    tpr = tp/(tp+fn)

    fpr = fp/(fp+tn)

    return fbeta, tpr, fpr

 

kfold = StratifiedKFold(n_splits= 5, shuffle=True, random_state = 42)

cvscores_SVM = []

for train, test in kfold.split(X, y):

    cvscores_SVM.append(mal_detection_SVM(Xtrain = X[train], Xtest = X[test], ytrain=  y[train], ytest = y[test]))

NameError: ignored

In [ ]:
pd.DataFrame(cvscores_SVM, columns= ['F-beta Score', 'TPR', "FPR"])

,F-beta Score,TPR,FPR


In [ ]:
import keras

from keras.models import Sequential

from keras.layers import Dense, Dropout, Activation

from keras import optimizers, regularizers

from sklearn import metrics

from keras.regularizers import l1, l2

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
def mal_detection_NN(Xtrain, Xtest, ytrain, ytest, num_epoch):

    mal_classifier = Sequential()

 

    # first hidden layer

    mal_classifier.add(Dense(units= 25, activation = 'relu',

                             activity_regularizer = l2(0.000001), input_dim = 215))

    mal_classifier.add(Dropout(0.2))

 

    #2nd Hidden layer

    mal_classifier.add(Dense(units= 15, activation = 'relu',

                             activity_regularizer = l2(0.000001)))

    mal_classifier.add(Dropout(0.2))

 

    #3rd hidden layer

 

    mal_classifier.add(Dense(units = 5, activation = 'relu',

                             activity_regularizer = l2(0.000001)))

    #output layer

    mal_classifier.add(Dense(units = 1, activation = 'sigmoid'))

    #defining optimizer

    adam = tf.keras.optimizers.Adam()

    #compiling model

    mal_classifier.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = ['accuracy'] )

 

    #defining callback to early stop monitoring on validation loss 

    monitor = EarlyStopping(monitor= "val_loss", min_delta = 1e-4, patience = 5, verbose = 0, mode = 'auto', 

                           restore_best_weights= True)

    callbacks_list = [monitor]

 

    # fitting the training data 

    history = mal_classifier.fit(Xtrain, ytrain, validation_data = (Xtest, ytest), callbacks = callbacks_list,

                                 epochs = num_epoch, batch_size = 64)

#     history = mal_classifier.fit(Xtrain, ytrain, validation_data = (Xtest, ytest), epochs = num_epoch, batch_size = 64)

 

    # predicting on test data 

    pred_prob = mal_classifier.predict(Xtest)

    pred = (pred_prob > 0.5 )

    #evaluating the model 

    fbeta = metrics.fbeta_score(ytest, pred, average='weighted', beta = 10)

    tn, fp, fn, tp = metrics.confusion_matrix(ytest, pred).ravel()

    tpr = tp/(tp+fn)

    fpr = fp/(fp+tn)

    

    

    # summarize history for loss

    plt.plot(history.history['loss'])

    plt.plot(history.history['val_loss'])

    plt.title('model loss')

    plt.ylabel('loss')

    plt.xlabel('epoch')

    plt.legend(['train', 'test'], loc='upper left')

    plt.show()

    return fbeta, tpr, fpr

In [ ]:
kfold = StratifiedKFold(n_splits= 5, shuffle=True, random_state = 42)

cvscores_NN = []

for train, test in kfold.split(X, y):

    cvscores_NN.append(mal_detection_NN(Xtrain = X[train], Xtest = X[test], ytrain=  y[train], ytest = y[test], num_epoch = 100 ))

NameError: ignored

In [1]:
fbeta_nn = pd.DataFrame([a for a,b,c in cvscores_NN], columns= ['Neural Netork'])

fbeta_svm = pd.DataFrame([a for a,b,c in cvscores_SVM], columns= ['SVM'])

fbeta_lr = pd.DataFrame([a for a,b,c in cvscores_LR], columns= ['LR'])

fbeta_all = pd.concat([fbeta_nn,  fbeta_svm, fbeta_lr], axis = 1)

fbeta_all

NameError: ignored